#### Summary:
In this notebook I will streamline the LAI HMM implementation to be easy to use in various test cases. *** NOTE: *** This notebook is currently designed to be implemented in the datahub team3 directory, not the github repo! This could certainly be streamlined even more, and will be when we test the effect of a range of recombination rates.

In [5]:
import random as rn
import numpy as np
import pandas as pd
import sys
import math

In [6]:
import datetime

In [7]:
from LAI_hmm_scriptFINAL import HMMOptimalPathLAI, HammingDist, makeSNPseq, standardizeIndices

In [8]:
from LAI_hmm_script_progress import HMMOptimalPathLAI_progress

# Read in Data and Create an HMM Object

For all simulated data tests:

In [5]:
#define names of the two test populations
test_PopA = "0"
test_PopB = "1"

In [6]:
#read in emissions data (simulated df)
emission_fp = "Git/cse284_project/Data/simData_N2_P100_seed518.tsv" #asumes in team3 dir
emission_df = pd.read_csv(emission_fp, sep='\t',header=0)

#renaming the columns (only necessary for simulated data)
colnames = ["POS","0_A","0_C","0_G","0_T","1_A","1_C","1_G","1_T"]
emission_df.columns = colnames
#emission_df.head()

Individual (test) specific:

In [5]:
#read in sample genotype to test
#assumes current dir is the team3 dir
genotype_fp = "Git/cse284_project/Data/simGenome_100_0_0.tsv"
genotype_df = pd.read_csv(genotype_fp, sep='\t',header = 0)
#genotype_df.head()

In [6]:
#cut down both dfs to only be positions shared in both
fin_emission_df, fin_genotype_df = standardizeIndices(emission_df,genotype_df,"POS")

In [7]:
fin_emission_df.head()

,POS,0_A,0_C,0_G,0_T,1_A,1_C,1_G,1_T
0,0,0.0000,0.3553,0.0000,0.6447,0.0000,0.5719,0.0000,0.4281
1,1,0.0000,0.7580,0.0000,0.2420,0.0000,0.4374,0.0000,0.5626
2,2,0.1303,0.0000,0.0000,0.8697,0.4271,0.0000,0.0000,0.5729
3,3,0.2157,0.0000,0.7843,0.0000,0.3534,0.0000,0.6466,0.0000
4,4,0.2444,0.0000,0.0000,0.7556,0.3655,0.0000,0.0000,0.6345


In [8]:
fin_genotype_df.head()

,POS,A1,A2,POP1,POP2
0,0,C,T,0,0
1,1,C,C,0,0
2,2,T,T,0,0
3,3,A,G,0,0
4,4,T,T,0,0


In [10]:
#create SNPseq
a1_idx = list(genotype_df.columns).index('A1')
a2_idx = list(genotype_df.columns).index('A2')
snps = makeSNPseq(fin_genotype_df,a1_idx,a2_idx)

HMM Creation and Initialization

In [11]:
#set recombination rate
recomb = 0.1

#create the hmm object
test_hmm00 = HMMOptimalPathLAI(test_PopA, test_PopB, fin_emission_df, recomb, snps)

#initialize the transition matrix
test_hmm00.get_transition_matrix()

# Reconstruct Haplotypes with the HMM Model

In [12]:
#perform the Viterbi algorithm
test_hmm00.get_optimal_path()
#print(test_hmm00.optimal_matrix)

#reconstruct the most probable path through states
test_hmm00.reconstruct_path()
#print(test_hmm00.final_path)

In [13]:
#convert the path to the desired output format (pass the string to use for CHR col)
path_df = test_hmm00.output_path("21")
#print(path_df.head())

#save to a file (also assumes are currently in team3 directory)
output_fp = "HMM_Test_Outputs/" + genotype_fp.split("/")[3][:-4] + "_HMMoutput.tsv"
print(output_fp)
path_df.to_csv(output_fp,sep='\t',header=True,index=False)

HMM_Test_Outputs/simGenome_100_0_0_HMMoutput.tsv


In [14]:
path_df.head()

,CHR,POS,POP1,POP2
0,21,0,0,0
1,21,1,0,0
2,21,2,0,0
3,21,3,0,0
4,21,4,0,0


In [ ]:
#alternative ultra streamlined version (create HMM --> get optimal path)

# recomb = 0.1
# test_hmm00 = HMMOptimalPathLAI(test_PopA, test_PopB, fin_emission_df, recomb, snps)
# test_hmm00.get_transition_matrix()
# test_hmm00.get_optimal_path()
# test_hmm00.reconstruct_path()
# path_df = test_hmm00.output_path("21")

# As One Function

In [5]:
#define names of the two test populations
test_PopA = "0"
test_PopB = "1"

In [6]:
#read in emissions data (simulated df)
emission_fp = "Git/cse284_project/Data/simData_N2_P100_seed518.tsv" #asumes in team3 dir
test_emission_df = pd.read_csv(emission_fp, sep='\t',header=0)

#renaming the columns (only necessary for simulated data)
colnames = ["POS","0_A","0_C","0_G","0_T","1_A","1_C","1_G","1_T"]
test_emission_df.columns = colnames
#emission_df.head()

In [9]:
def HMM_implementation_wrapper(popA, popB, genotype_fp, emission_df, recomb, output_fp, chrm):
    #read in sample genotype to test -- assumes current dir is the team3 dir
    genotype_df = pd.read_csv(genotype_fp, sep='\t',header = 0)

    #cut down both dfs to only be positions shared in both
    fin_emission_df, fin_genotype_df = standardizeIndices(emission_df, genotype_df, "POS")
    
    #create SNPseq
    a1_idx = list(fin_genotype_df.columns).index('A1')
    a2_idx = list(fin_genotype_df.columns).index('A2')
    snps = makeSNPseq(fin_genotype_df,a1_idx,a2_idx)
    
    #create the hmm object
    hmm = HMMOptimalPathLAI(popA, popB, fin_emission_df, recomb, snps)
    hmm.get_transition_matrix()
    
    #perform the Viterbi algorithm and reconstruct the most probable path through states
    hmm.get_optimal_path()
    hmm.reconstruct_path()

    #convert the path to the desired output format (pass the string to use for CHR col)
    path_df = hmm.output_path(chrm)
    path_df.to_csv(output_fp,sep='\t',header=True,index=False)

# Running the HMM on Short Simulated Genotypes with Various Recombination Rates

In [22]:
%%bash

#collect all the file names in one file
#touch /home/hmummey/teams/CSE284_SP21_A00/team3/HMM_Test_Outputs/simulated_files.txt
cd /home/hmummey/teams/CSE284_SP21_A00/team3/Git/cse284_project/Data

echo /home/hmummey/teams/CSE284_SP21_A00/team3/Git/cse284_project/Data >> /home/hmummey/teams/CSE284_SP21_A00/team3/HMM_Test_Outputs/simulated_files.txt
ls simAd*.tsv >> /home/hmummey/teams/CSE284_SP21_A00/team3/HMM_Test_Outputs/simulated_files.txt
ls simGen*.tsv >> /home/hmummey/teams/CSE284_SP21_A00/team3/HMM_Test_Outputs/simulated_files.txt

In [8]:
%%bash 
head -n 20 /home/hmummey/teams/CSE284_SP21_A00/team3/HMM_Test_Outputs/simulated_files.txt

/home/hmummey/teams/CSE284_SP21_A00/team3/Git/cse284_project/Data
simAdmixedGenome_100_0_1_Rx1_0.tsv
simAdmixedGenome_100_0_1_Rx1_1.tsv
simAdmixedGenome_100_0_1_Rx2_0.tsv
simAdmixedGenome_100_0_1_Rx2_1.tsv
simAdmixedGenome_100_0_1_Rx4_0.tsv
simAdmixedGenome_100_0_1_Rx4_1.tsv
simAdmixedGenome_100_0_1_Rx5_0.tsv
simAdmixedGenome_100_0_1_Rx5_1.tsv
simGenome_100_0_0.tsv
simGenome_100_0_1.tsv
simGenome_100_1_0.tsv
simGenome_100_1_1.tsv
simGenome_perfect_100_0_0.tsv
simGenome_perfect_100_0_1.tsv
simGenome_perfect_100_1_0.tsv
simGenome_perfect_100_1_1.tsv


In [9]:
#open the file with the path and names, then loop through and run the HMM and save the output

simulated_fp = "~/teams/CSE284_SP21_A00/team3/HMM_Test_Outputs/simulated_files.txt"
f = open(simulated_fp, 'r')

for line in f:
    if "/home" in line:
        direct = line.split("\n")[0]
        #print(direct)
    else:
        fp = direct + "/" + line.split("\n")[0]
        print(fp)
        
        #testing readin
        #test = pd.read_csv(fp,sep='\t')
        #print(test.head(1))
        if "Admixed" in fp:
            for recombr in [0.001,0.01,0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]:
                output_fp = "~/teams/CSE284_SP21_A00/team3/HMM_Test_Outputs/" + line.split(".")[0] + "_recomb"+ str(recombr) + "_HMMoutput.tsv"
                #print(output_fp)
                HMM_implementation_wrapper(test_PopA, test_PopB, fp, test_emission_df, recombr, output_fp, "21")
    
        else:
            recombr = 0.1
            output_fp = "~/teams/CSE284_SP21_A00/team3/HMM_Test_Outputs/" + line.split(".")[0] + "_recomb"+ str(recombr) + "_HMMoutput.tsv"
            #print(output_fp)
            HMM_implementation_wrapper(test_PopA, test_PopB, fp, test_emission_df, recombr, output_fp, "21")
    
f.close()

/home/hmummey/teams/CSE284_SP21_A00/team3/Git/cse284_project/Data/simAdmixedGenome_100_0_1_Rx1_0.tsv
/home/hmummey/teams/CSE284_SP21_A00/team3/Git/cse284_project/Data/simAdmixedGenome_100_0_1_Rx1_1.tsv
/home/hmummey/teams/CSE284_SP21_A00/team3/Git/cse284_project/Data/simAdmixedGenome_100_0_1_Rx2_0.tsv
/home/hmummey/teams/CSE284_SP21_A00/team3/Git/cse284_project/Data/simAdmixedGenome_100_0_1_Rx2_1.tsv
/home/hmummey/teams/CSE284_SP21_A00/team3/Git/cse284_project/Data/simAdmixedGenome_100_0_1_Rx4_0.tsv
/home/hmummey/teams/CSE284_SP21_A00/team3/Git/cse284_project/Data/simAdmixedGenome_100_0_1_Rx4_1.tsv
/home/hmummey/teams/CSE284_SP21_A00/team3/Git/cse284_project/Data/simAdmixedGenome_100_0_1_Rx5_0.tsv
/home/hmummey/teams/CSE284_SP21_A00/team3/Git/cse284_project/Data/simAdmixedGenome_100_0_1_Rx5_1.tsv
/home/hmummey/teams/CSE284_SP21_A00/team3/Git/cse284_project/Data/simGenome_100_0_0.tsv
/home/hmummey/teams/CSE284_SP21_A00/team3/Git/cse284_project/Data/simGenome_100_0_1.tsv
/home/hmummey/te

# Medium Length Genomes (5/31/21)

In [10]:
#define names of the two test populations
test_PopA = "0"
test_PopB = "1"

In [11]:
#read in emissions data (simulated df)
#emission_fp = "simulated_files/simData_N2_P5000_seed531.tsv" #assumes in team3 dir
emission_fp = "simulated_files/simData_N2_P5000_seed531_formatted.tsv"
emission_df5000 = pd.read_csv(emission_fp, sep='\t',header=0)

#renaming the columns (only necessary for simulated data)
colnames = ["POS","0_A","0_C","0_G","0_T","1_A","1_C","1_G","1_T"]
emission_df5000.columns = colnames
emission_df5000.head()

,POS,0_A,0_C,0_G,0_T,1_A,1_C,1_G,1_T
0,0,0.000001,0.556700,0.443300,0.000001,0.000001,0.648700,0.351300,0.000001
1,1,0.461900,0.000001,0.000001,0.538100,0.288000,0.000001,0.000001,0.712000
2,2,0.000001,0.842500,0.157500,0.000001,0.000001,0.334700,0.665300,0.000001
3,3,0.274400,0.000001,0.725600,0.000001,0.954200,0.000001,0.045800,0.000001
4,4,0.000001,0.000001,0.397200,0.602800,0.000001,0.000001,0.492600,0.507400


In [9]:
%%bash

#collect all the file names in one file
touch HMM_Test_Outputs/5kSNP_files.txt
cd simulated_files

echo simulated_files >> ../HMM_Test_Outputs/5kSNP_files.txt
ls simAdmixedGenome_5000*.tsv >> ../HMM_Test_Outputs/5kSNP_files.txt

In [12]:
%%bash
# Should only have 10 files
head -n 15 HMM_Test_Outputs/5kSNP_files.txt

simAdmixedGenome_5000_0_1_Rx10_0.tsv
simAdmixedGenome_5000_0_1_Rx10_1.tsv
simAdmixedGenome_5000_0_1_Rx1_0.tsv
simAdmixedGenome_5000_0_1_Rx1_1.tsv
simAdmixedGenome_5000_0_1_Rx2_0.tsv
simAdmixedGenome_5000_0_1_Rx2_1.tsv
simAdmixedGenome_5000_0_1_Rx4_0.tsv
simAdmixedGenome_5000_0_1_Rx4_1.tsv
simAdmixedGenome_5000_0_1_Rx5_0.tsv
simAdmixedGenome_5000_0_1_Rx5_1.tsv


In [13]:
#open the file with the path and names, then loop through and run the HMM and save the output

simulated_fp = "HMM_Test_Outputs/5kSNP_files.txt"
f = open(simulated_fp, 'r')

for line in f:
    if "/home" in line:
        direct = line.split("\n")[0]
        #print(direct)
    else:
        #fp = direct + "/" + line.split("\n")[0]
        fp="simulated_files/" + line.split("\n")[0]
        print(fp)
        
        #recomb rates: 1/10000, 1/7500, 1/5000, 1/2500, 1/1000, 1/500, 1/100
        #recomb_rates = [0.00013, 0.0001, 0.0002, 0.0004, 0.001, 0.002, 0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
        #recomb_rates = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
        recomb_rates = [0.00013, 0.0001, 0.0002, 0.0004, 0.001, 0.002, 0.01]
        for recombr in recomb_rates:
            #print(str(recombr)[:7])
            output_fp = "HMM_Test_Outputs/" + line.split(".")[0] + "_recomb"+ str(recombr)[:7] + "_HMMoutput.tsv"
            #print(output_fp)
            HMM_implementation_wrapper(test_PopA, test_PopB, fp, emission_df5000, recombr, output_fp, "21")
    
f.close()


simulated_files/simAdmixedGenome_5000_0_1_Rx10_0.tsv
simulated_files/simAdmixedGenome_5000_0_1_Rx10_1.tsv
simulated_files/simAdmixedGenome_5000_0_1_Rx1_0.tsv
simulated_files/simAdmixedGenome_5000_0_1_Rx1_1.tsv
simulated_files/simAdmixedGenome_5000_0_1_Rx2_0.tsv
simulated_files/simAdmixedGenome_5000_0_1_Rx2_1.tsv
simulated_files/simAdmixedGenome_5000_0_1_Rx4_0.tsv
simulated_files/simAdmixedGenome_5000_0_1_Rx4_1.tsv
simulated_files/simAdmixedGenome_5000_0_1_Rx5_0.tsv
simulated_files/simAdmixedGenome_5000_0_1_Rx5_1.tsv


In [19]:
#running it on one Rx4 5k file and getting the emissions matrix

#read in sample genotype to test -- assumes current dir is the team3 dir
genotype_df = pd.read_csv("simulated_files/simAdmixedGenome_5000_0_1_Rx4_1.tsv", sep='\t',header = 0)

#cut down both dfs to only be positions shared in both
fin_emission_df, fin_genotype_df = standardizeIndices(emission_df5000, genotype_df, "POS")

#create SNPseq
a1_idx = list(fin_genotype_df.columns).index('A1')
a2_idx = list(fin_genotype_df.columns).index('A2')
snps = makeSNPseq(fin_genotype_df,a1_idx,a2_idx)

#create the hmm object
recomb = 1/5000
hmm = HMMOptimalPathLAI(test_PopA, test_PopB, fin_emission_df, recomb, snps)
hmm.get_transition_matrix()

#perform the Viterbi algorithm and reconstruct the most probable path through states
hmm.get_optimal_path()
probs_df = hmm.optimal_matrix

hmm.reconstruct_path()

In [20]:
#convert the path to the desired output format (pass the string to use for CHR col)
path_df = hmm.output_path("21")
#path_df.to_csv(output_fp,sep='\t',header=True,index=False)

In [33]:
probs_df2 = pd.DataFrame(probs_df)
probs_df2[735]

0    4.006872e-321
1    9.896542e-317
2     0.000000e+00
3    1.403146e-321
Name: 735, dtype: float64

In [16]:
probs_df2.to_csv("simAdmixedGenome_5000_0_1_Rx4_1_Probabilities.tsv",sep='\t')

# Running the HMM on Long Simulated Admixed Genotypes

In [4]:
popA = "AFR"
popB = "EUR"

Emissions files

In [5]:
#chr14
chr14_emissions_fp = "/home/hmummey/teams/CSE284_SP21_A00/team3/chromosome_14_files/chr14_genotypes_afr_eur_allelefreqs.bybp.csv"
emission_df_chr14 = pd.read_csv(chr14_emissions_fp, sep=',',header=0)
emission_df_chr14.head()

,CHR,POS,SNP,A2,A1,MAF_AFR,MAF_EUR,AFR_A,AFR_C,AFR_G,AFR_T,EUR_A,EUR_C,EUR_G,EUR_T
0,14,19000017,rs375700886,C,T,0.0000,0.00000,0.000001,1.000000,0.000001,0.000001,0.000001,1.000000,0.000001,0.000001
1,14,19000050,rs543746158,G,A,0.0000,0.00000,0.000001,0.000001,1.000000,0.000001,0.000001,0.000001,1.000000,0.000001
2,14,19000056,rs561973970,A,T,0.0000,0.00000,1.000000,0.000001,0.000001,0.000001,1.000000,0.000001,0.000001,0.000001
3,14,19000059,rs201622908,G,T,0.0000,0.00498,0.000001,0.000001,1.000000,0.000001,0.000001,0.000001,0.995020,0.004980
4,14,19000060,rs28973059,C,G,0.1188,0.38840,0.000001,0.881200,0.118800,0.000001,0.000001,0.611600,0.388400,0.000001


In [6]:
#chr21
chr21_emissions_fp = "/home/hmummey/teams/CSE284_SP21_A00/team3/chromosome_21_files/chr21_genotypes_afr_eur_allelefreqs.bybp.csv"
emission_df_chr21 = pd.read_csv(chr21_emissions_fp, sep=',',header=0)
emission_df_chr21.head()

,CHR,POS,SNP,A2,A1,MAF_AFR,MAF_EUR,AFR_A,AFR_C,AFR_G,AFR_T,EUR_A,EUR_C,EUR_G,EUR_T
0,21,9411239,rs559462325,G,A,0.000000,0.000000,0.000001,0.000001,1.000000,0.000001,0.000001,0.000001,1.000000,0.000001
1,21,9411245,rs181691356,C,A,0.000893,0.001992,0.000893,0.999107,0.000001,0.000001,0.001992,0.998008,0.000001,0.000001
2,21,9411264,rs548263598,A,C,0.000893,0.000000,0.999107,0.000893,0.000001,0.000001,1.000000,0.000001,0.000001,0.000001
3,21,9411267,rs561987868,G,T,0.000000,0.000000,0.000001,0.000001,1.000000,0.000001,0.000001,0.000001,1.000000,0.000001
4,21,9411302,rs531010746,G,T,0.011610,0.000000,0.000001,0.000001,0.988390,0.011610,0.000001,0.000001,1.000000,0.000001


Genotype files

In [46]:
# %%bash

# #collect all the file names in one file
# touch /home/hmummey/teams/CSE284_SP21_A00/team3/HMM_Test_Outputs/admixed_files.txt
# cd /home/hmummey/teams/CSE284_SP21_A00/team3/simulated_files

# echo /home/hmummey/teams/CSE284_SP21_A00/team3/simulated_files >> /home/hmummey/teams/CSE284_SP21_A00/team3/HMM_Test_Outputs/admixed_files.txt
# ls admix*.tsv >> /home/hmummey/teams/CSE284_SP21_A00/team3/HMM_Test_Outputs/admixed_files.txt

In [7]:
%%bash 
head -n 20 /home/hmummey/teams/CSE284_SP21_A00/team3/HMM_Test_Outputs/admixed_files.txt

/home/hmummey/teams/CSE284_SP21_A00/team3/simulated_files
admixEUR_AFR_chr14_Rx1_a.tsv
admixEUR_AFR_chr14_Rx1_b.tsv
admixEUR_AFR_chr14_Rx1_c.tsv
admixEUR_AFR_chr14_Rx3_a.tsv
admixEUR_AFR_chr14_Rx3_b.tsv
admixEUR_AFR_chr14_Rx3_c.tsv
admixEUR_AFR_chr21_Rx1_a.tsv
admixEUR_AFR_chr21_Rx1_b.tsv
admixEUR_AFR_chr21_Rx1_c.tsv
admixEUR_AFR_chr21_Rx3_a.tsv
admixEUR_AFR_chr21_Rx3_b.tsv
admixEUR_AFR_chr21_Rx3_c.tsv
admixEUR_Rx1a_chr21_Rx1_a.tsv
admixEUR_Rx1a_chr21_Rx1_b.tsv
admixEUR_Rx1a_chr21_Rx1_c.tsv
admixRx1c_Rx3b_chr21_Rx1_a.tsv
admixRx1c_Rx3b_chr21_Rx1_b.tsv
admixRx1c_Rx3b_chr21_Rx1_c.tsv


In [69]:
# #open the file with the path and names, then loop through and run the HMM and save the output

# admixed_fp = "/home/hmummey/teams/CSE284_SP21_A00/team3/HMM_Test_Outputs/admixed_files.txt"
# f = open(admixed_fp, 'r')

# for line in f:
#     if "/home" in line:
#         direct = line.split("\n")[0]
#         #print(direct)
#     else:
#         fp = direct + "/" + line.split("\n")[0]
#         print(fp)
#         now = datetime.datetime.now()
#         print (now.strftime("%Y-%m-%d %H:%M:%S"))
        
#         #testing readin
#         #test = pd.read_csv(fp,sep='\t')
#         #print(test.head(1))
        
#         output_fp = "/home/hmummey/teams/CSE284_SP21_A00/team3/HMM_Test_Outputs/" + line.split(".")[0] + "_HMMoutput.tsv"
#         #print(output_fp)
        
#         if "14" in fp:
#             HMM_implementation_wrapper(popA, popB, fp, emission_df_chr14, 0.1, output_fp, "14")
#         elif "21" in fp:
#             HMM_implementation_wrapper(popA, popB, fp, emission_df_chr21, 0.1, output_fp, "21")
        
    
# f.close()

/home/hmummey/teams/CSE284_SP21_A00/team3/simulated_files/admixEUR_AFR_chr14_Rx1_a.tsv
2021-05-26 20:18:00


KeyError: 682

# Troubleshooting on Long Simulated Admixed (one genome)

In [7]:
print(len(emission_df_chr14))
emission_df_chr14.head()

2505787


,CHR,POS,SNP,A2,A1,MAF_AFR,MAF_EUR,AFR_A,AFR_C,AFR_G,AFR_T,EUR_A,EUR_C,EUR_G,EUR_T
0,14,19000017,rs375700886,C,T,0.0000,0.00000,0.000001,1.000000,0.000001,0.000001,0.000001,1.000000,0.000001,0.000001
1,14,19000050,rs543746158,G,A,0.0000,0.00000,0.000001,0.000001,1.000000,0.000001,0.000001,0.000001,1.000000,0.000001
2,14,19000056,rs561973970,A,T,0.0000,0.00000,1.000000,0.000001,0.000001,0.000001,1.000000,0.000001,0.000001,0.000001
3,14,19000059,rs201622908,G,T,0.0000,0.00498,0.000001,0.000001,1.000000,0.000001,0.000001,0.000001,0.995020,0.004980
4,14,19000060,rs28973059,C,G,0.1188,0.38840,0.000001,0.881200,0.118800,0.000001,0.000001,0.611600,0.388400,0.000001


In [8]:
#go through manually (step by step) for the first file: /home/hmummey/teams/CSE284_SP21_A00/team3/simulated_files/admixEUR_AFR_chr14_Rx1_a.tsv
admixed_fp1 = "/home/hmummey/teams/CSE284_SP21_A00/team3/simulated_files/admixEUR_AFR_chr14_Rx1_a.tsv"
genotype_df2 = pd.read_csv(admixed_fp1, sep='\t',header = 0)
print(len(genotype_df2))
genotype_df2.head()

2539145


,POS,A1,A2,POP1,POP2
0,19000017,T,T,1,0
1,19000050,A,A,1,0
2,19000056,T,T,1,0
3,19000059,T,T,1,0
4,19000060,G,G,1,0


In [9]:
fin_emission_df2, fin_genotype_df2 = standardizeIndices(emission_df_chr14, genotype_df2, "POS")
print(len(fin_emission_df2))
fin_emission_df2.head()

2500619


,CHR,POS,SNP,A2,A1,MAF_AFR,MAF_EUR,AFR_A,AFR_C,AFR_G,AFR_T,EUR_A,EUR_C,EUR_G,EUR_T
0,14,19000017,rs375700886,C,T,0.0000,0.00000,0.000001,1.000000,0.000001,0.000001,0.000001,1.000000,0.000001,0.000001
1,14,19000050,rs543746158,G,A,0.0000,0.00000,0.000001,0.000001,1.000000,0.000001,0.000001,0.000001,1.000000,0.000001
2,14,19000056,rs561973970,A,T,0.0000,0.00000,1.000000,0.000001,0.000001,0.000001,1.000000,0.000001,0.000001,0.000001
3,14,19000059,rs201622908,G,T,0.0000,0.00498,0.000001,0.000001,1.000000,0.000001,0.000001,0.000001,0.995020,0.004980
4,14,19000060,rs28973059,C,G,0.1188,0.38840,0.000001,0.881200,0.118800,0.000001,0.000001,0.611600,0.388400,0.000001


In [10]:
print(len(fin_genotype_df2))
fin_genotype_df2.head()

2500619


,POS,A1,A2,POP1,POP2
0,19000017,T,T,1,0
1,19000050,A,A,1,0
2,19000056,T,T,1,0
3,19000059,T,T,1,0
4,19000060,G,G,1,0


In [13]:
#create SNPseq
a1_idx = list(fin_genotype_df2.columns).index('A1')
a2_idx = list(fin_genotype_df2.columns).index('A2')
print(a1_idx,a2_idx)
now = datetime.datetime.now()
print (now.strftime("%Y-%m-%d %H:%M:%S"))

snps = makeSNPseq(fin_genotype_df2,a1_idx,a2_idx)
print(snps[:10])
now = datetime.datetime.now()
print (now.strftime("%Y-%m-%d %H:%M:%S"))

1 2
2021-05-27 11:45:16
['TT', 'AA', 'TT', 'TT', 'GG', 'AA', 'CC', 'GG', 'GG', 'TT']
2021-05-27 11:47:33


In [14]:
print(len(snps))

2500619


In [15]:
recomb = 0.1

In [16]:
#create the hmm object
now = datetime.datetime.now()
print (now.strftime("%Y-%m-%d %H:%M:%S"))

hmm = HMMOptimalPathLAI(popA, popB, fin_emission_df2, recomb, snps)
hmm.get_transition_matrix()
now = datetime.datetime.now()
print (now.strftime("%Y-%m-%d %H:%M:%S"))

#perform the Viterbi algorithm and reconstruct the most probable path through states
hmm.get_optimal_path()
hmm.reconstruct_path()
now = datetime.datetime.now()
print (now.strftime("%Y-%m-%d %H:%M:%S"))

#convert the path to the desired output format (pass the string to use for CHR col)
path_df = hmm.output_path("14")
path_df.to_csv(output_fp,sep='\t',header=True,index=False)
now = datetime.datetime.now()
print (now.strftime("%Y-%m-%d %H:%M:%S"))

2021-05-27 11:47:33
2021-05-27 11:47:33


KeyboardInterrupt: 

# Debugging

Figuring out weird indexing issue

In [50]:
print(list(fin_genotype_df2.index)[:1000] == list(range(1,1000)))
print(list(fin_genotype_df2.index)[:1000])

False
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220

In [52]:
set1 = set(list(fin_genotype_df2.index)[:1000])
set2 = set(range(1,1000))
#print(set1)
#print(set2)
print(len(set1.intersection(set2)))
print(set1 - set1.intersection(set2))
print(set2 - set1.intersection(set2))

996
{0, 1000, 1002, 1001}
{908, 973, 390}


In [17]:
list(fin_emission_df2["POS"]) == list(fin_genotype_df2["POS"])

True

In [55]:
fin_emission_df2.head()

,CHR,POS,SNP,A2,A1,MAF_AFR,MAF_EUR,AFR_A,AFR_C,AFR_G,AFR_T,EUR_A,EUR_C,EUR_G,EUR_T
0,14,19000017,rs375700886,C,T,0.0000,0.00000,0.000001,1.000000,0.000001,0.000001,0.000001,1.000000,0.000001,0.000001
1,14,19000050,rs543746158,G,A,0.0000,0.00000,0.000001,0.000001,1.000000,0.000001,0.000001,0.000001,1.000000,0.000001
2,14,19000056,rs561973970,A,T,0.0000,0.00000,1.000000,0.000001,0.000001,0.000001,1.000000,0.000001,0.000001,0.000001
3,14,19000059,rs201622908,G,T,0.0000,0.00498,0.000001,0.000001,1.000000,0.000001,0.000001,0.000001,0.995020,0.004980
4,14,19000060,rs28973059,C,G,0.1188,0.38840,0.000001,0.881200,0.118800,0.000001,0.000001,0.611600,0.388400,0.000001


In [56]:
fin_emission_df2.iloc[682,:]

CHR                 14
POS           19013506
SNP        rs559727658
A2                   A
A1                   G
MAF_AFR       0.000000
MAF_EUR       0.000000
AFR_A         1.000000
AFR_C         0.000001
AFR_G         0.000001
AFR_T         0.000001
EUR_A         1.000000
EUR_C         0.000001
EUR_G         0.000001
EUR_T         0.000001
Name: 683, dtype: object

In [59]:
fin_emission_df2.iloc[389,:]

CHR                14
POS          19007198
SNP        rs28838603
A2                  A
A1                  T
MAF_AFR      0.000000
MAF_EUR      0.022910
AFR_A        1.000000
AFR_C        0.000001
AFR_G        0.000001
AFR_T        0.000001
EUR_A        0.977090
EUR_C        0.000001
EUR_G        0.000001
EUR_T        0.022910
Name: 389, dtype: object

Testing pandas.DF.reset_index

In [12]:
fin_emission_df3, fin_genotype_df3 = standardizeIndices(emission_df_chr14, genotype_df2, "POS")
print(len(fin_emission_df3))
fin_emission_df3.tail()

2500619


,index,CHR,POS,SNP,A2,A1,MAF_AFR,MAF_EUR,AFR_A,AFR_C,AFR_G,AFR_T,EUR_A,EUR_C,EUR_G,EUR_T
2500614,2505782,14,107289436,rs28971294,C,G,0.155400,0.3904,0.000001,0.844600,0.155400,0.000001,0.000001,0.609600,0.390400,0.000001
2500615,2505783,14,107289442,rs572272149,A,G,0.001786,0.0000,0.998214,0.000001,0.001786,0.000001,1.000000,0.000001,0.000001,0.000001
2500616,2505784,14,107289452,rs140896985,C,T,0.001786,0.0000,0.000001,0.998214,0.000001,0.001786,0.000001,1.000000,0.000001,0.000001
2500617,2505785,14,107289453,rs149770569,G,A,0.000000,0.0000,0.000001,0.000001,1.000000,0.000001,0.000001,0.000001,1.000000,0.000001
2500618,2505786,14,107289456,rs540661577,C,T,0.000000,0.0000,0.000001,1.000000,0.000001,0.000001,0.000001,1.000000,0.000001,0.000001


In [13]:
fin_emission_df3.reset_index(inplace=True,drop=True)
fin_emission_df3.tail()

,index,CHR,POS,SNP,A2,A1,MAF_AFR,MAF_EUR,AFR_A,AFR_C,AFR_G,AFR_T,EUR_A,EUR_C,EUR_G,EUR_T
2500614,2505782,14,107289436,rs28971294,C,G,0.155400,0.3904,0.000001,0.844600,0.155400,0.000001,0.000001,0.609600,0.390400,0.000001
2500615,2505783,14,107289442,rs572272149,A,G,0.001786,0.0000,0.998214,0.000001,0.001786,0.000001,1.000000,0.000001,0.000001,0.000001
2500616,2505784,14,107289452,rs140896985,C,T,0.001786,0.0000,0.000001,0.998214,0.000001,0.001786,0.000001,1.000000,0.000001,0.000001
2500617,2505785,14,107289453,rs149770569,G,A,0.000000,0.0000,0.000001,0.000001,1.000000,0.000001,0.000001,0.000001,1.000000,0.000001
2500618,2505786,14,107289456,rs540661577,C,T,0.000000,0.0000,0.000001,1.000000,0.000001,0.000001,0.000001,1.000000,0.000001,0.000001


Testing different indexing strategies too

In [69]:
print(fin_emission_df3.columns)
em_idx1 = list(fin_emission_df3.columns).index("AFR_A")
print(em_idx1)

Index(['index', 'CHR', 'POS', 'SNP', 'A2', 'A1', 'MAF_AFR', 'MAF_EUR', 'AFR_A',
       'AFR_C', 'AFR_G', 'AFR_T', 'EUR_A', 'EUR_C', 'EUR_G', 'EUR_T'],
      dtype='object')
8
